In [1]:
import autogen

from autogen import AssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

import nest_asyncio
nest_asyncio.apply()


config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    file_location="..",
    filter_dict={
        "model": ["gpt-3.5-turbo", "gpt-35-turbo", "gpt-35-turbo-0613", "gpt-4", "gpt4"],
    },
)

print("LLM models: ", [config_list[i]["model"] for i in range(len(config_list))])

# Termination message definition
termination_msg = (
    lambda x: isinstance(x, dict)
    and str(x.get("content", "")).upper() == "TERMINATE"
)




LLM models:  ['gpt-4']


In [2]:
llm_config = {
    "config_list": config_list,  # config_list should be defined or imported
    "timeout": 120,
    "seed": 42,
}

In [3]:
from typing import Callable, Literal, Optional, Union, Dict
from autogen.agentchat.conversable_agent import ConversableAgent
from typing_extensions import Annotated


def run_meta_prompting(expert_name: Annotated[str, "Name of the expert. Should follow the format: Expert xxx."], 
                   expert_identity: Annotated[str, "A high-quality description about the most capable and suitable expert to answer the instruction. In second person perspective. For example, You are a linguist, well-versed in the study of language and its structures. You are equipped with a good understanding of grammar rules and can differentiate between nouns, verbs, adjectives, adverbs, etc. You can quickly and accurately identify the parts of speech in a sentence and explain the role of each word in the sentence. Your expertise in language and grammar is highly valuable in analyzing and understanding the nuances of communication."], 
                   task: Annotated[str, "The task that needs to be solved by the expert."]) -> str:
    """
    Run Meta-prompting to solve the task.
    The method is adapted from "Meta-Prompting: Enhancing Language Models with Task-Agnostic Scaffolding".
    Paper available at https://arxiv.org/abs/2401.12954
    """
    print("Running meta prompting...")
    print("Querying expert: ", expert_name)

    expert = autogen.AssistantAgent(
        name=expert_name,
        human_input_mode="NEVER",
        llm_config=llm_config,
        system_message='You are an AI assistant that helps people find information. Please answer the following question. Once you have determined the final answer, please present it using the format below:\n\n>> FINAL ANSWER:\n"""\n[final answer]\n"""',
        max_consecutive_auto_reply=1,
    )
    user_proxy = autogen.UserProxyAgent(
        name="user_proxy",
        human_input_mode="NEVER",
        default_auto_reply="TERMINATE",
        code_execution_config={
            "work_dir": "coding",
            "use_docker": False,
        },
        max_consecutive_auto_reply=1,
    )
    task += "\nYou have access to python code interpreter. Suggest python code block starting with '```python' and the code will be automatically executed. You can use code to solve the task or for result verification. You should always use print statement to get the value of a variable."
    user_proxy.initiate_chat(expert, message=expert_identity + "\n" + task, silent=True)

    expert_reply = user_proxy.chat_messages[expert][1]["content"]
    proxy_reply = user_proxy.chat_messages[expert][2]["content"]

    if proxy_reply != "TERMINATE":
        code_result = proxy_reply[proxy_reply.find("Code output:") + len("Code output:") :].strip()
        expert_reply += f"\nThis is the output of the code blocks when executed:\n{code_result}"
    else:
        expert_reply.replace(
            "FINAL ANSWER:",
            f"{expert_name}'s final answer:\n",
        )

    return expert_reply


class MetaAgent(ConversableAgent):
    SYSTEM_MESSAGE = """You are Meta-Expert, an extremely clever expert with the unique ability to collaborate with multiple experts (such as Expert Problem Solver, Expert Mathematician, Expert Essayist, etc.) to tackle any task and solve any complex problems. Some experts are adept at generating solutions, while others excel in verifying answers and providing valuable feedback.
As Meta-Expert, your role is to oversee the communication between the experts, effectively using their skills to answer a given question while applying your own critical thinking and verification abilities.
To communicate with a expert, call function "meta_prompting" with the expert's name, identity information and the task that needs to be solved. The function will return a response from the expert.
Ensure that your instructions are clear and unambiguous, and include all necessary information within the triple quotes. You should assign personas to the experts (e.g., "You are a physicist specialized in...").
You can interact with only one expert at a time, and break complex problems into smaller, solvable tasks. Each interaction is treated as an isolated event, so include all relevant details in every call.
Refrain from repeating the very same questions to experts. Examine their responses carefully and seek clarification if required, keeping in mind they don't recall past interactions.
Upon the completion of all tasks and verifications, you should conclude the result and reply "TERMINATE".
"""
    # TOOL = {
    #     "type": "function",
    #     "function": {
    #         "name": "meta_prompting",
    #         "description": "Solve a task by querying an expert. Provide the expert identity and the task that needs to be solved, and the function will return the response of the expert.",
    #         "parameters": {
    #             "type": "object",
    #             "properties": {
    #                 "task": {
    #                     "type": "string",
    #                     "description": "[REQUIRED] The task that needs to be solved by the expert.",
    #                 },
    #                 "expert_name": {
    #                     "type": "string",
    #                     "description": "[REQUIRED] Name of the expert. Should follow the format: Expert xxx.",
    #                 },
    #                 "expert_identity": {
    #                     "type": "string",
    #                     "description": "[REQUIRED] A high-quality description about the most capable and suitable expert to answer the instruction. In second person perspective. For example, You are a linguist, well-versed in the study of language and its structures. You are equipped with a good understanding of grammar rules and can differentiate between nouns, verbs, adjectives, adverbs, etc. You can quickly and accurately identify the parts of speech in a sentence and explain the role of each word in the sentence. Your expertise in language and grammar is highly valuable in analyzing and understanding the nuances of communication.",
    #                 },
    #             },
    #         },
    #     },
    # }

    def __init__(
        self,
        name: str,
        system_message: Optional[str] = None,
        llm_config: Optional[Union[Dict, Literal[False]]] = None,
        is_termination_msg: Optional[Callable[[Dict], bool]] = None,
        max_consecutive_auto_reply: Optional[int] = None,
        human_input_mode: Optional[str] = "NEVER",
        code_execution_config: Optional[Union[Dict, Literal[False]]] = False,
        description: Optional[
            str
        ] = "A helpful AI assistant that can build a group of agents at a proper time to solve a task.",
        **kwargs,
    ):
        super().__init__(
            name=name,
            system_message=self.SYSTEM_MESSAGE,
            llm_config=llm_config,
            is_termination_msg=is_termination_msg,
            max_consecutive_auto_reply=max_consecutive_auto_reply,
            human_input_mode=human_input_mode,
            code_execution_config=code_execution_config,
            description=description,
            **kwargs,
        )
        # self.update_tool_signature(self.TOOL, is_remove=False)




In [4]:
run_meta_prompting(**{
  "task": "Please provide me with the latest research or review articles related to the modeling of reliability and safety mechanisms in AI systems, with a focus on Large Language Models.",
  "expert_name": "Expert Researcher",
  "expert_identity": "You are an expert researcher with access to the latest academic databases and the ability to provide summaries of the most recent articles from Arxiv. Your expertise is in parsing through large volumes of academic literature to identify relevant papers, especially those pertaining to the fields of artificial intelligence, machine learning, and their applications. You are up-to-date with current trends and advancements in AI, and you can quickly discern which articles will be most useful for constructing a comprehensive blog post on a given topic."
})

Running meta prompting...
Querying expert:  Expert Researcher


'As an AI language model, I do not have real-time access to external databases like Arxiv or the ability to execute Python code. However, I can guide you on how to find the latest research articles related to the modeling of reliability and safety mechanisms in AI systems, especially Large Language Models (LLMs).\n\nTo find the latest articles on Arxiv, one would typically follow these steps:\n\n1. Visit the Arxiv website (https://arxiv.org/).\n2. Use the search functionality to find papers related to your topic of interest.\n   - You can use keywords such as "reliability in AI systems," "safety mechanisms in AI," "Large Language Models safety," etc.\n3. Filter the results by selecting the "Sort by Submission Date" to get the most recent submissions.\n4. Review the abstracts of papers that seem relevant to your topic.\n5. Download the full papers for a detailed understanding of the proposed models and approaches.\n\nSince I\'m unable to perform a live search for you, I can still provid

In [5]:
llm_config = {
    "config_list": config_list,  # config_list should be defined or imported
    "timeout": 120,
    "seed": 42,
}

In [6]:
# Example usage:
import autogen.runtime_logging


PROBLEM = "Write a comperhansive blog post about the modelling of reliability and safety mechanisms in AI system. You should get update knowledge from Arxiv before start writing. The focus should be on Large Language Models."

proxy = autogen.UserProxyAgent(
    name="proxy",
    human_input_mode="NEVER",
    code_execution_config=False,
    max_consecutive_auto_reply=1,
    default_auto_reply="Continue. If you think the task is solved, please reply me only with 'TERMINATE'.",
)


agent = MetaAgent(
    name="Meta-Expert",
    llm_config=llm_config,
    human_input_mode="NEVER",
    max_consecutive_auto_reply=15,
)



In [7]:
# agent.llm_config['tools']

In [8]:
autogen.agentchat.register_function(
    run_meta_prompting,
    caller=agent,
    executor=proxy,
    description="Solve a task by querying an expert. Provide the expert identity and the task that needs to be solved, and the function will return the response of the expert.",
    )

agent.llm_config['tools']

[{'type': 'function',
  'function': {'description': 'Solve a task by querying an expert. Provide the expert identity and the task that needs to be solved, and the function will return the response of the expert.',
   'name': 'run_meta_prompting',
   'parameters': {'type': 'object',
    'properties': {'expert_name': {'type': 'string',
      'description': 'Name of the expert. Should follow the format: Expert xxx.'},
     'expert_identity': {'type': 'string',
      'description': 'A high-quality description about the most capable and suitable expert to answer the instruction. In second person perspective. For example, You are a linguist, well-versed in the study of language and its structures. You are equipped with a good understanding of grammar rules and can differentiate between nouns, verbs, adjectives, adverbs, etc. You can quickly and accurately identify the parts of speech in a sentence and explain the role of each word in the sentence. Your expertise in language and grammar is hi

In [9]:
proxy.register_function(function_map={"meta_prompting": lambda **args: run_meta_prompting(**args)})
proxy.function_map

{'run_meta_prompting': <function __main__.run_meta_prompting(expert_name: typing.Annotated[str, 'Name of the expert. Should follow the format: Expert xxx.'], expert_identity: typing.Annotated[str, 'A high-quality description about the most capable and suitable expert to answer the instruction. In second person perspective. For example, You are a linguist, well-versed in the study of language and its structures. You are equipped with a good understanding of grammar rules and can differentiate between nouns, verbs, adjectives, adverbs, etc. You can quickly and accurately identify the parts of speech in a sentence and explain the role of each word in the sentence. Your expertise in language and grammar is highly valuable in analyzing and understanding the nuances of communication.'], task: typing.Annotated[str, 'The task that needs to be solved by the expert.']) -> str>,
 'meta_prompting': <function __main__.<lambda>(**args)>}

In [11]:
# Start logging
# logging_session_id = autogen.runtime_logging.start(config={"dbname": "logs.db"})
# print("Logging session ID: " + str(logging_session_id))

proxy.initiate_chat(agent, message=PROBLEM)

# End logging
# autogen.runtime_logging.stop()

proxy (to Meta-Expert):

Write a comperhansive blog post about the modelling of reliability and safety mechanisms in AI system. You should get update knowledge from Arxiv before start writing. The focus should be on Large Language Models.

--------------------------------------------------------------------------------
Meta-Expert (to proxy):

***** Suggested tool Call (call_mPBAmHzWNGuOfJLGffi0hy1Z): run_meta_prompting *****
Arguments: 
{
  "expert_name": "Expert Researcher",
  "expert_identity": "You are a researcher with up-to-date knowledge and access to scientific articles, particularly from repositories such as Arxiv. You possess the skill to quickly scan and summarize recent literature, especially related to reliability and safety mechanisms in AI systems.",
  "task": "Please provide a summary of the most recent advancements or discussions from Arxiv on the topic of reliability and safety mechanisms in AI systems, focusing specifically on Large Language Models."
}
**************

/home/autogen/autogen/autogen/agentchat/conversable_agent.py:1121: UserWarning: Cannot extract summary using last_msg: 'NoneType' object has no attribute 'replace'. Using an empty str as summary.
  warnings.warn(f"Cannot extract summary using last_msg: {e}. Using an empty str as summary.", UserWarning)


ChatResult(chat_id=None, chat_history=[{'content': 'Write a comperhansive blog post about the modelling of reliability and safety mechanisms in AI system. You should get update knowledge from Arxiv before start writing. The focus should be on Large Language Models.', 'role': 'assistant'}, {'tool_calls': [{'id': 'call_mPBAmHzWNGuOfJLGffi0hy1Z', 'function': {'arguments': '{\n  "expert_name": "Expert Researcher",\n  "expert_identity": "You are a researcher with up-to-date knowledge and access to scientific articles, particularly from repositories such as Arxiv. You possess the skill to quickly scan and summarize recent literature, especially related to reliability and safety mechanisms in AI systems.",\n  "task": "Please provide a summary of the most recent advancements or discussions from Arxiv on the topic of reliability and safety mechanisms in AI systems, focusing specifically on Large Language Models."\n}', 'name': 'run_meta_prompting'}, 'type': 'function'}], 'content': None, 'role':